This Colab uses Meterstick to analyze churn rates on a fake A/B testing data.

In [ ]:
!git clone https://github.com/google/meterstick.git
import sys, os
sys.path.append(os.getcwd())

Cloning into 'meterstick'...
remote: Enumerating objects: 1060, done.
remote: Counting objects: 100% (385/385), done.
remote: Compressing objects: 100% (153/153), done.
remote: Total 1060 (delta 264), reused 245 (delta 232), pack-reused 675 (from 3)
Receiving objects: 100% (1060/1060), 2.20 MiB | 4.23 MiB/s, done.
Resolving deltas: 100% (727/727), done.


In [ ]:
import numpy as np
import pandas as pd
from meterstick import *

In [ ]:
pd.set_option('future.no_silent_downcasting', True)

In [ ]:
# @title Create fake data
np.random.seed(42)
n = 1000
churn_rates = {'control': 0.2, 'treatment1': 0.1, 'treatment2': 0.25, 'treatment3': 0.3}
df = pd.DataFrame({
    'region': np.random.choice(('US', 'EU'), n),
    'experiment': np.random.choice(list(churn_rates), n),
})
# Make churn rates at the US 0.03 higher to those at EU.
df['lost'] = np.random.random(n) < df.experiment.replace(churn_rates) + 0.03 * (df.region == 'US')
df.head()

<ipython-input-3-309128abeb5c>:10: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['lost'] = np.random.random(n) < df.experiment.replace(churn_rates) + 0.03 * (df.region == 'US')


,region,experiment,lost
0,US,treatment1,False
1,EU,treatment2,False
2,US,control,False
3,US,control,False
4,US,control,False


In [ ]:
churn = (Sum('lost') / Count('lost')).set_name('churn')
churn.compute_on(df, ['region', 'experiment'])

churn
region experiment          
EU     control     0.181818
       treatment1  0.103175
       treatment2  0.250000
       treatment3  0.230769
US     control     0.290323
       treatment1  0.189781
       treatment2  0.296296
       treatment3  0.340426

In [ ]:
churn_change = churn | PercentChange('experiment', 'control')
churn_change.compute_on(df, 'region')

churn Percent Change
region experiment                      
EU     treatment1            -43.253968
       treatment2             37.500000
       treatment3             26.923077
US     treatment1            -34.630981
       treatment2              2.057613
       treatment3             17.257683

In [ ]:
bs_res = churn_change | Bootstrap(confidence=0.95) | compute_on(df, 'region')
bs_res

Metric            churn Percent Change                                      
                                 Value Bootstrap CI-lower Bootstrap CI-upper
region experiment                                                           
EU     treatment1           -43.253968         -81.613881          -4.894055
       treatment2            37.500000         -31.416674         106.416674
       treatment3            26.923077         -40.400804          94.246958
US     treatment1           -34.630981         -65.102206          -4.159756
       treatment2             2.057613         -38.056615          42.171842
       treatment3            17.257683         -30.326610          64.841976

In [ ]:
bs_res.display()

Dimensions,churn
controlEU,0.1818
treatment1EU,"0.1032-43.25%[-81.61, -4.89] %"
treatment2EU,"0.250037.50%[-31.42, 106.42] %"
treatment3EU,"0.230826.92%[-40.40, 94.25] %"
controlUS,0.2903
treatment1US,"0.1898-34.63%[-65.10, -4.16] %"
treatment2US,"0.29632.06%[-38.06, 42.17] %"
treatment3US,"0.340417.26%[-30.33, 64.84] %"
